In [1]:
from SBM_SDE_tensor import *
from obs_and_flow_classes_and_functions1 import *
#import seaborn as sns
import torch
from torch import nn
import torch.distributions as D
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
from tqdm import tqdm
import random
from torch.autograd import Function
import argparse
import os
import sys
from pathlib import Path
import shutil
import pandas as pd

In [2]:
torch.manual_seed(0)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

cuda_id = 1
dt = .2 #.2 #SDE discretization timestep.
t = 100 #200 #Simulation run for T hours.
n = int(t / dt) 
t_span = np.linspace(0, t, n + 1)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n + 1, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
pretrain_lr = 1e-3
elbo_lr = 1e-3
niter = 2 #50
piter = 1
batch_size = 3 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.
prior_scale_factor = 0.25 #Prior standard deviations set at 0.25 of prior means.
obs_error_scale_factor = 0.25

In [3]:
temp_ref = 283

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1

SCON_C_prior_means = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}
SCON_SS_prior_means = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC}

#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_C_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_SS_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

#System parameters from deterministic model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#Diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_ECA_C_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_ECA_SS_prior_means = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [4]:
#Obtain SOC and DOC pool litter inputs for all SBMs.
i_s_tensor = 0.001 + 0.0005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous SOC input function
i_d_tensor = 0.0001 + 0.00005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous DOC input function

In [5]:
#Mean field VI code block.

#Define mean-field class: consumes parameter dictionary with values used as initial mean values.
class MeanField(nn.Module):
    def __init__(self, init_params, sdev_scale_factor):
        super().__init__()

        #Use param dict to intialise the means for the mean-field approximations.
        means = []
        keys = []
        for key, value in init_params.items():
            keys += [key]
            means += [value]
        self.means = nn.Parameter(torch.Tensor(means))
        self.sds = nn.Parameter(self.means * sdev_scale_factor)
        #Save keys for forward output.
        self.keys = keys

    def forward(self, n = 30):
        #Update posterior.
        q_dist = D.normal.Normal(LowerBound.apply(self.means, 1e-6), LowerBound.apply(self.sds, 1e-8))
        #q_dist = D.log_normal.LogNormal(LowerBound.apply(self.means, 1e-6), LowerBound.apply(self.sds, 1e-8)) #Testing LogNormal sampling object.
        #Sample theta ~ q(theta).
        samples = LowerBound.apply(q_dist.rsample([n]), 1e-6)
        #samples_log = LowerBound.apply(q_dist.rsample([n]), 1e-6)
        #samples = torch.exp(samples_log) #Exp transformation of log sample SDE theta values for SDE use.
        #Evaluate log prob of theta samples.
        log_q_theta = torch.sum(q_dist.log_prob(samples), -1) #Shape of n.
        #log_q_theta = torch.sum(q_dist.log_prob(samples_log), -1) #Shape of n.
        #Return samples in same dictionary format.
        dict_out = {} #Define dictionary with n samples for each parameter.
        for key, sample in zip(self.keys, torch.split(samples, 1, -1),):
            dict_out[f"{key}"] = sample.squeeze(1) #Each sample is of shape [n].
        #Return samples in dictionary and tensor format.
        return dict_out, samples, log_q_theta

In [6]:
def calc_log_lik(C_PATH, T_SPAN_TENSOR, DT, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF):
    drift, diffusion_sqrt = DRIFT_DIFFUSION(C_PATH[:, :-1, :], T_SPAN_TENSOR[:, :-1, :], I_S_TENSOR[:, :-1, :], I_D_TENSOR[:, :-1, :], PARAMS_DICT, TEMP_GEN, TEMP_REF)
    print('\nDrift = ', drift)
    print('\nDiffusion = ', diffusion_sqrt)
    euler_maruyama_state_sample_object = D.multivariate_normal.MultivariateNormal(loc = C_PATH[:, :-1, :] + drift * DT, scale_tril = diffusion_sqrt * math.sqrt(DT))
    return euler_maruyama_state_sample_object.log_prob(C_PATH[:, 1:, :]).sum(-1)

In [7]:
def train(DEVICE, PRETRAIN_LR, ELBO_LR, NITER, PRETRAIN_ITER, BATCH_SIZE,
          PRIOR_SCALE_FACTOR, SDEFLOW, ObsModel, csv_to_obs_df, DATA_CSV,
          OBS_ERROR_SCALE_FACTOR, STATE_DIM, T, DT, N, T_SPAN_TENSOR,
          I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAM_PRIOR_MEANS_DICT,
          TEMP_GEN, TEMP_REF, ANALYTICAL_STEADY_STATE_INIT,
          LR_DECAY=0.1, DECAY_STEP_SIZE=100000, LEARN_THETA=False, PRINT_EVERY=10):
    if PRETRAIN_ITER >= NITER:
        raise Exception("PRETRAIN_ITER must be < NITER.")
    
    # Load data
    obs_times, obs_means, obs_error = csv_to_obs_df(DATA_CSV, STATE_DIM + 1, T, OBS_ERROR_SCALE_FACTOR) #Import data CSV and extract observation times, means, and desired observation error standard deviation based on `obs_error_scale_factor`. 
    
    # Initialize models
    obs_model = ObsModel(DEVICE, obs_times, DT, obs_means[:-1, :], obs_error[:, :-1]) #Hack for bypassing ObsModel and SDEFlow dimension mismatch issue.
    net = SDEFlow(DEVICE, obs_model, STATE_DIM, T, DT, N,
                  I_S_TENSOR, I_D_TENSOR, cond_inputs=3).to(DEVICE) #Instantiate flow.
    prior_means_tensor = torch.Tensor(list(PARAM_PRIOR_MEANS_DICT.values())) #Convert prior mean dictionary values to tensor.
    
    #Establish approximate priors.
    theta_dict = {k: torch.tensor(v).expand(BATCH_SIZE) for k, v in PARAM_PRIOR_MEANS_DICT.items()}
    if LEARN_THETA:
        priors = D.normal.Normal(prior_means_tensor, prior_means_tensor * PRIOR_SCALE_FACTOR)
        q_theta = MeanField(PARAM_PRIOR_MEANS_DICT, PRIOR_SCALE_FACTOR)
    else:
        q_theta = None
    
    #Initiate optimizers.
    pretrain_optimizer = optim.Adam(net.parameters(), lr = PRETRAIN_LR, eps = 1e-7)
    elbo_params = list(net.parameters()) + list(q_theta.parameters()) if LEARN_THETA \
        else net.parameters()
    ELBO_optimizer = optim.Adamax(elbo_params, lr = ELBO_LR)
    
    # Record loss throughout training
    best_loss_norm = 1e10
    best_loss_ELBO = 1e20
    norm_losses = [] #[best_loss_norm] * 10
    ELBO_losses = [] #[best_loss_ELBO] * 10
    
    #Joint optimization of SDE and hidden (NN) parameters loop.
    with tqdm(total = NITER, desc = f'\nTrain Diffusion', position = -1) as tq:
        for it in range(NITER):
            net.train()
            C_PATH, log_prob = net(BATCH_SIZE) #Obtain paths with solutions at times after t0.
            
            if LEARN_THETA:
                theta_dict, theta, log_q_theta = q_theta(BATCH_SIZE)
                print('\ntheta_dict = ', theta_dict)
            else:
                log_q_theta = torch.tensor(0.)
                
            C_0 = LowerBound.apply(ANALYTICAL_STEADY_STATE_INIT(I_S_TENSOR[0, 0, 0].item(), I_D_TENSOR[0, 0, 0].item(), theta_dict), 1e-7) #Calculate deterministic initial conditions.
            print('\nC_0 =', C_0)
            #C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(DEVICE) #Commenting out because analytical steady state init functions now output tensors with appropriate batch size if argument into MeanField forward function is BATCH_SIZE. #Assign initial conditions to C_PATH.
            C_PATH = torch.cat([C_0.unsqueeze(1), C_PATH], 1) #Append deterministic CON initial conditions conditional on parameter values to C path. 
            print('\nC_PATH =', C_PATH)
            print('\nC_PATH mean =', C_PATH.mean(-2))
            
            if it <= PRETRAIN_ITER:
                pretrain_optimizer.zero_grad()
                #l1_norm_element = C_PATH - torch.mean(obs_model.mu, -1)
                #l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                #best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                #norm_losses.append(l1_norm.item())
                l2_norm_element = C_PATH - torch.mean(obs_model.mu, -1)
                l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l2_norm.item())
                
                if it % PRINT_EVERY == 0:
                    ma_norm_loss = sum(norm_losses[-10:]) / len(norm_losses[-10:])
                    print(f"\nMoving average norm loss at {iter} iterations is: {ma_norm_loss}. Best norm loss value is: {best_loss_norm}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\nC_PATH =', C_PATH)
                #l1_norm.backward()
                l2_norm.backward()
                pretrain_optimizer.step()
            else:
                ELBO_optimizer.zero_grad()
                
                if LEARN_THETA:
                    log_p_theta = priors.log_prob(theta).sum(-1)
                else:
                    log_p_theta = torch.tensor(0.)
                
                log_lik = calc_log_lik(C_PATH, T_SPAN_TENSOR.to(DEVICE), dt,
                                       I_S_TENSOR.to(DEVICE), I_D_TENSOR.to(DEVICE),
                                       DRIFT_DIFFUSION, theta_dict, TEMP_GEN, TEMP_REF)
                neg_ELBO = -log_p_theta.mean() + log_q_theta.mean() - log_lik.mean() - \
                    obs_model(C_PATH, theta_dict) + log_prob.mean() #From equation 14 of Ryder et al., 2019.
                #neg_ELBO = -log_lik.mean() - obs_model(C_PATH) + log_prob.mean() #Old ELBO computation without joint density optimization.
                print('\nneg_ELBO_mean = ', neg_ELBO)
                best_loss_ELBO = neg_ELBO if neg_ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(neg_ELBO)
                
                if it % PRINT_EVERY == 0:
                    ma_elbo_loss = sum(ELBO_losses[-10:]) / len(ELBO_losses[-10:])
                    print(f"\nMoving average ELBO loss at {iter} iterations is: {ma_elbo_loss}. Best ELBO loss value is: {best_loss_ELBO}.")
                neg_ELBO.backward()
                ELBO_optimizer.step()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            if it % DECAY_STEP_SIZE == 0 and it > 0:
                ELBO_optimizer.param_groups[0]['lr'] *= LR_DECAY
            tq.update()
            
    return net, q_theta, ELBO_losses, norm_losses

In [8]:
train(devi, pretrain_lr, elbo_lr, niter, piter, batch_size, prior_scale_factor, SDEFlow, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_prior_means, temp_gen, temp_ref, analytical_steady_state_init_CON)



Train Diffusion:   0%|          | 0/2 [00:00<?, ?it/s]


C_0 = tensor([[45.6603,  0.0715,  0.7147],
        [45.6603,  0.0715,  0.7147],
        [45.6603,  0.0715,  0.7147]])

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 0.7676,  1.4146,  1.3784],
         [ 0.5256,  0.9502,  1.5853],
         ...,
         [ 1.9552,  0.8116,  0.4268],
         [ 0.9608,  0.9149,  0.6179],
         [ 0.7719,  0.8116,  0.7150]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.7676,  0.6944,  0.6549],
         [ 3.0924,  0.8305,  0.5890],
         ...,
         [ 0.7685,  0.5667,  0.5120],
         [ 0.8255,  0.6553,  0.6401],
         [ 0.8018,  0.4106,  1.0328]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.7676,  0.7949,  0.6166],
         [ 2.0430,  1.4852,  0.6336],
         ...,
         [ 0.7302,  0.7095,  0.5555],
         [ 0.7441,  0.5122,  0.5824],
         [ 0.7835,  0.8573,  0.7085]]], grad_fn=<CatBackward>)

C_PATH mean = tensor([[1.1556, 0.8601, 0.8378],
        [1.1391, 0.9664, 0.7997],
        [1.0846, 0.8930, 0.8258]]



Train Diffusion:  50%|█████     | 1/2 [00:01<00:01,  1.73s/it]


C_0 = tensor([[45.6603,  0.0715,  0.7147],
        [45.6603,  0.0715,  0.7147],
        [45.6603,  0.0715,  0.7147]])

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 1.0578,  0.8548,  0.6041],
         [ 0.9383,  0.6376,  2.8327],
         ...,
         [ 0.8345,  0.4921,  0.3800],
         [ 0.5765,  2.1464,  0.6323],
         [ 1.4628,  0.5088,  0.9314]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.8910,  1.0638,  1.1144],
         [ 2.3308,  1.0274,  0.5371],
         ...,
         [ 2.0195,  2.3989,  0.2233],
         [ 1.6336,  0.5005,  1.0384],
         [ 0.8202,  0.6163,  0.7794]],

        [[45.6603,  0.0715,  0.7147],
         [ 1.1549,  0.7920,  0.4633],
         [ 4.5558,  1.8253,  0.4626],
         ...,
         [ 2.6075,  0.4797,  0.4723],
         [ 1.1347,  0.4749,  0.6764],
         [ 1.8384,  1.0347,  1.0175]]], grad_fn=<CatBackward>)

C_PATH mean = tensor([[1.8207, 1.0403, 0.6830],
        [2.0908, 1.0111, 0.6675],
        [1.8768, 1.0077, 0.6733]]



Train Diffusion: 100%|██████████| 2/2 [00:03<00:00,  1.80s/it]


(SDEFlow(
   (obs_model): ObsModel()
   (coupling): ModuleList(
     (0): CouplingLayer(
       (feature_net): Sequential(
         (0): ResNetBlockUnMasked(
           (conv1): Conv1d(6, 96, kernel_size=(3,), stride=(1,), padding=(1,))
           (conv2): Conv1d(96, 96, kernel_size=(3,), stride=(1,), padding=(1,))
           (act1): PReLU(num_parameters=96)
           (act2): PReLU(num_parameters=96)
           (bn1): Identity()
           (bn2): Identity()
           (conv_skip): Conv1d(6, 96, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
         )
         (1): ResNetBlockUnMasked(
           (conv1): Conv1d(96, 6, kernel_size=(3,), stride=(1,), padding=(1,))
           (conv2): Conv1d(6, 6, kernel_size=(3,), stride=(1,), padding=(1,))
           (act1): PReLU(num_parameters=6)
           (act2): PReLU(num_parameters=6)
           (bn1): Identity()
           (bn2): Identity()
           (conv_skip): Conv1d(96, 6, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)


In [ ]:
t_span_tensor.size()

In [ ]:
prior_means_tensor = torch.Tensor(list(SCON_C_prior_means.values()))
priors = D.normal.Normal(prior_means_tensor, prior_means_tensor * prior_scale_factor)
q_theta = MeanField(SCON_C_prior_means, prior_scale_factor)
theta_dict, theta, log_q_theta = q_theta(batch_size)
print(log_q_theta)

In [ ]:
theta_dict

In [ ]:
print(priors.log_prob(theta))
log_p_theta_1 = priors.log_prob(theta).sum(-1)
print(log_p_theta_1)
log_p_theta_2 = priors.log_prob(theta).sum()
print(log_p_theta_2)
#log_p_theta_1 is correct way of computing log_p_theta

In [ ]:
obs_times, obs_means, obs_error = csv_to_obs_df('CON_synthetic_sol_df.csv', 3 + 1, t, obs_error_scale_factor)
obs_model = ObsModel(devi, obs_times, dt, obs_means[:-1, :], obs_error[:, :-1])
net = SDEFlow(devi, batch_size, obs_model, 3, t, dt, n).to(devi)

In [ ]:
C_PATH, log_prob = net()
print(C_PATH)
C_PATH.size()

In [ ]:
C_0 = analytical_steady_state_init_CON(i_s_tensor[0, 0, 0].item(), i_d_tensor[0, 0, 0].item(), theta_dict)
print(C_0)
C_0.unsqueeze(1).size()

In [ ]:
C_PATH = torch.cat([C_0.unsqueeze(1), C_PATH], 1)
C_PATH

In [ ]:
SOC, DOC, MBC =  torch.chunk(C_PATH, 3, -1)
SOC.size()

In [ ]:
current_temp = temp_gen(t_span_tensor, temp_ref)

In [ ]:
theta_dict['k_S_ref'].size()

In [ ]:
k_S = arrhenius_temp_dep(theta_dict['k_S_ref'], current_temp, theta_dict['Ea_S'], temp_ref)
k_S = k_S.permute(2, 1, 0)
k_D = arrhenius_temp_dep(theta_dict['k_D_ref'], current_temp, theta_dict['Ea_D'], temp_ref)
k_D = k_D.permute(2, 1, 0)
k_M = arrhenius_temp_dep(theta_dict['k_M_ref'], current_temp, theta_dict['Ea_M'], temp_ref)
k_M = k_M.permute(2, 1, 0)

In [ ]:
theta_dict_repeat = dict((k, v.repeat(1, 2501, 1).permute(2, 1, 0)) for k, v in theta_dict.items())
drift_SOC = i_s_tensor + theta_dict_repeat['a_DS'] * k_D * DOC + theta_dict_repeat['a_M'] * theta_dict_repeat['a_MSC'] * k_M * MBC - k_S * SOC
drift_DOC = i_d_tensor + theta_dict_repeat['a_SD'] * k_S * SOC + theta_dict_repeat['a_M'] * (1 - theta_dict_repeat['a_MSC']) * k_M * MBC - (theta_dict_repeat['u_M'] + k_D) * DOC
drift_MBC = theta_dict_repeat['u_M'] * DOC - k_M * MBC

In [ ]:
drift_MBC.size()

In [ ]:
theta_dict_u_M_test = theta_dict['u_M'].repeat(1, 2501, 1)
theta_dict_u_M_test = theta_dict_u_M_test.permute(2, 1, 0)
print(theta_dict_u_M_test.size())
test = theta_dict_u_M_test * DOC
print(test.size())

In [ ]:
C_PATH.size()

In [ ]:
drift = torch.empty_like(C_PATH, device = C_PATH.device) #Initiate tensor with same dims as C_PATH to assign drift.
drift[:, :, 0 : 1] = drift_SOC
drift[:, :, 1 : 2] = drift_DOC
drift[:, :, 2 : 3] = drift_MBC

In [ ]:
test_m = torch.stack([theta_dict['c_SOC'], theta_dict['c_DOC'], theta_dict['c_MBC']], 1)
print(LowerBound.apply(test_m, 1e-6))
test_m_sqrt = torch.sqrt(test_m)
torch.diag_embed(test_m_sqrt)

In [ ]:
diffusion_sqrt_single = torch.diag_embed(torch.sqrt(LowerBound.apply(torch.stack([theta_dict['c_SOC'], theta_dict['c_DOC'], theta_dict['c_MBC']], 1), 1e-6)))
diffusion_sqrt_single

In [ ]:
diffusion_sqrt_single.unsqueeze(1).expand(-1, 2501, -1, -1)

In [ ]:
a = torch.arange(3).reshape((1, 3))
a

In [ ]:
b = a[(None, )*2] # same as a.reshape((1, 1, -1))
b

In [ ]:
b.shape

In [ ]:
a.repeat((2, 1))

In [ ]:
torch.tensor(3).shape